In [1]:
# The APF wavelength solution we have been using is not quite accurate. 
# Howard has provided another one (which is still not perefeclty accurate but much better - see explore_wl_soln.ipynb). 

# We could fix this by re-running everything with the new wavelength solution. But, since SM-Emp first shifts the spectrum into the stellar
# rest frame before matching, the shifted spectrum and residual should be on correct scales. So, we only need to fix the wavelength scale 
# for the unshifted spectrum, then redefine the velocity shift values.

# In fact, this should only need to be done once: the unshifted spectra are all on the same wavelength scale. No that's wrong. 
# We need to do this for each target, because the spectrum itself will be different (essentially the start and endpoints that are mapped
# to the start and end of the reference wl scale will be different). 


In [1]:
import import_ipynb
from bstar_deblaze import bstar_deblazed2 #ADZ ADD 7/17/20
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.io.fits as fits
from rescale import get_rescaled_wave_soln
#from rescale import resample_order
import sys
import os
# Specmatch imports (relative to current directory)
sys.path.insert(0, '/mnt_home/azuckerman/BL_APF_DAP/specmatch_emp')
from specmatch_emp.specmatchemp import library
from specmatch_emp.specmatchemp import plots as smplot
from specmatch_emp.specmatchemp.spectrum import Spectrum
from specmatch_emp.specmatchemp.specmatch import SpecMatch
new_apf_wave_soln = (fits.open('apf_wave_2022.fits'))[0].data

# define new resampling function (resample each order individually, and then combine)
def new_resample(wave_soln_ref, wave_soln, spectrum):
    resampled_orders = np.empty(shape=(0)) 
    resampled_wl = np.empty(shape=(0)) # contains resampled wl for each order concatenated (so there are repeats)
    for order in np.arange(30,52,1):
        order_values = spectrum[order][100:-100] # truncate the ends becuase the deblazing doesn't work well here --> unless in future iteration do this in deblazing
        apf_wl_values = wave_soln[order][100:-100]
        first = apf_wl_values[0]
        last = apf_wl_values[-1]
        new_first = first - 0.017 
        new_last = last + 0.17 
        new_wl_section = np.array(wave_soln_ref)[(new_first <= wave_soln_ref) * (new_last >= wave_soln_ref)]
        resampled_order = resample_order(new_wl_section, apf_wl_values, order_values)
        resampled_orders = np.hstack([resampled_orders, resampled_order])
        resampled_wl = np.hstack([resampled_wl, new_wl_section])
    # average duplicate flux values
    resampled_spectrum = np.zeros(len(wave_soln_ref))
    i=0
    for wl in wave_soln_ref:
        values = resampled_orders[resampled_wl == wl]
        avg = np.nanmean(values)
        resampled_spectrum[i] = avg
        i += 1       
    return resampled_spectrum

# define new resampling function for baseline photons -> sum not average in overlap regions
def new_resample_baseline(wave_soln_ref, wave_soln, arr):
    resampled_orders = np.empty(shape=(0)) 
    resampled_wl = np.empty(shape=(0)) # contains resampled wl for each order concatenated (so there are repeats)
    for order in np.arange(30,52,1):
        #order_values = arr[order][100:-101] # truncate the ends becuase the deblazing doesn't work well here --> unless in future iteration do this in deblazing
        #apf_wl_values = wave_soln[order][100:-100]
        order_values = arr[order][:-1] 
        apf_wl_values = wave_soln[order][:-1]
        first = apf_wl_values[0]
        last = apf_wl_values[-1]
        new_first = first - 0.017 
        new_last = last + 0.17 
        new_wl_section = np.array(wave_soln_ref)[(new_first <= wave_soln_ref) * (new_last >= wave_soln_ref)]
        resampled_order = resample_order(new_wl_section, apf_wl_values, order_values)
        resampled_orders = np.hstack([resampled_orders, resampled_order])
        resampled_wl = np.hstack([resampled_wl, new_wl_section])
    # sum duplicate flux values
    resampled_arr = np.zeros(len(wave_soln_ref))
    i=0
    for wl in wave_soln_ref:
        values = resampled_orders[resampled_wl == wl]
        total = np.nansum(values)
        resampled_arr[i] = total
        i += 1       
    return resampled_arr

def resample_order(wave_data_new, wave_data_old, data):
    # MODIFED VERSION!!
    import numpy as np

    w_lst = wave_data_old
    s_lst = data

    # create array that has wavelengths in column 0 and intensity in column 1
    ws = np.zeros((len(w_lst), 2))
    for i in range(len(w_lst)):
        ws[i,0] = w_lst[i]
        ws[i,1] = s_lst[i]

    # sort the array by increasing wavelength
    # the result of merging spectral orders does not strictly increase
    # because orders overlap in wavelength ranges
    # ws = ws[ws[:, 0].argsort()]

    # select to skip first 137 values (closer to 4997 Å) - via eye test
    # ws = ws[137:]
    
    # average intensity values within +/- 0.017 Å on the new wavelength scale
    
    # comparisons between wavelengths in the old and new scale start from the
    # beginning of the old wavelength array for each new wavelength value
    # inefficient, so data points in the front of the array are clipped every
    # 1000 comparisons to reduce time wasted on wavelength comparisons
    data_new = []
    count = 0
    pairs = ws.tolist() # pairs -> (old wavelength, old intensity)
    for w in wave_data_new: # for every new wavelength value
        if (count != 0):
            if ((count % 1000 == 0)): # for every thousand comparisons

                # remove 800 pairs from the beginning of the
                # list of old values
                removed_1 = pairs[:800]
                removed_1.reverse()
                del pairs[:800]

                # if we remove too many,
                # the new wavelength value is less than the
                # starting wavelength value in the old list
                # i.e. (w_new - w_old < 0)
                # so we return some of the pairs
                if (w - pairs[0][0] < 0):
                    for pair in removed_1:
                        pairs.insert(0, pair)
                        break # currently breaks after inserting one pair

                # while the new wavelength value is greater than
                # the starting wavelength in the old list by more than a tenth,
                # remove a pair
                # (to improve efficiency, avoiding unneccessary comparisons)
                while ((w - pairs[0][0] > .1)):
                    removed_2 = pairs[0]
                    del pairs[0]
                    if (w - pairs[0][0] < 0): # replace if (w_new - w_old < 0)
                        pairs.insert(0, removed_2)
                        break
        avg_lst = []
        some_found = False
        for pair in pairs:
            # comparison process between new and old wavelength values
            if (abs(w - pair[0]) < .017):
                avg_lst.append(pair[1])
                some_found = True
            elif (some_found): break # break when the inequality no longer holds
        data_new.append(np.mean(np.asarray(avg_lst)))
        count += 1

        
    #print('Finished resample') #ADZ DELETE THIS 6/23/20
    return data_new

# Get a wavelength solution rescaled onto the scale of the library
wave_soln_ref = get_rescaled_wave_soln()

# library
lib = library.read_hdf()

importing Jupyter notebook from bstar_deblaze.ipynb
importing Jupyter notebook from rescale.ipynb
Finished rescale
Reading library from /home/azuckerman/.specmatchemp/library.h5


In [ ]:
#  Get spectra filelist
path_to_dir = './APF_spectra/all_individual_spectra_clean' # change for each run
run_type = 'ind' # all_apf, all_obs, or ind
filelist = os.listdir(path_to_dir)
# for running on a subset of the files

try:
    filelist.remove('.ipynb_checkpoints') # remove hidden file in this directory
    filelist.remove('HIP5643_spectra') # remove problematic spectrum; produces an error but not due to labeling (GJ54.1)
except ValueError:
    pass

photon_counts_path =  'SM_photon_counts/' + run_type # first rename SM_photon_counts to SM_photon_coutns_old_wl_scale 
unshifted_out_path =  'APF_spectra/NDRU_' + run_type # first rename APF_spectra/NDRU_[run_type] to  APF_spectra/NDRU_[run_type]_old_wl_scale


for dirname in filelist[2:]:
    
    print(dirname)
    
    
    sim_name = dirname.split('_')[0] #filename.replace('_spectra', '') # read simbad resolvable name from directory name
    # define name for saving shifted spectrum, residual, plots, etc to distinguish files in the case that there are multiple
    # subdirectories for the same star in this directory (NOTE: in that case must follow naming structure star_spectra_number
    # for each subdirectory)
    if dirname.endswith('spectra'):
        id_name = sim_name
    elif dirname[-1].isdigit():
        id_name = sim_name + '_' + dirname.split('_')[-1]
    if len(os.listdir(path_to_dir + '/' + dirname)) < 1:
        print('Skipping ' + dirname + ' due to empty directory.')
        empty_dirs += [dirname]
        continue    

    path_name = str(path_to_dir) + '/' + dirname       
    filenames = [f for f in os.listdir(path_name) if os.path.isfile(os.path.join(path_name, f))] 

    
    # Sum all of the data files for a star
    data = np.zeros((79, 4608))
    counter = 0
    for filename in filenames:
        file = fits.open(path_name + '/' + filename)
        data_part = file[0].data
        if (str(np.shape(data_part)) != '(79, 4608)'):
            print(str(np.shape(data_part)) + ' is an incompatible data shape.')
            print('Cannot perform shift-and-match process.')
            sys.exit()
        if counter == 0: #ADZ 7/26/20: get the header from the first file for this star, to use for the residual fits file 
            use_header = file[0].header
        counter += 1
        try:
            data += data_part
        except ValueError:
            ve = True
            
    data_deblaze = np.copy(data)
    data_no_deblaze = np.copy(data)
    for order_inc in range(22):
        data_deblaze[30 + order_inc, :4600], ignore1, ignore2, ingore3 = bstar_deblazed2(data, 30 + order_inc)  
        

    # Resample the spectrum onto the new wavelength scale
    new_data_resamp = new_resample(wave_soln_ref, new_apf_wave_soln, data_deblaze)
    
    # resample the raw photon values without deblazing
    resamp_no_deblaze = new_resample_baseline(wave_soln_ref, new_apf_wave_soln, data_no_deblaze)
    
    # save array of absolute fluxes values
    save_photon_counts = True
    if save_photon_counts:
        photons_file = photon_counts_path + '/photon_counts_' + id_name + '.csv'
        photons_df = pd.DataFrame(resamp_no_deblaze)
        photons_df.to_csv(photons_file, index = False)

    # Create spectrum object
    my_spectrum = Spectrum(np.asarray(wave_soln_ref), np.asarray(new_data_resamp))
    my_spectrum.name = sim_name

    #lib = specmatchemp.library.read_hdf() ADZ 8/10/20 moved this to outer loop so can remove stars from library
    
    sm = SpecMatch(my_spectrum, lib)
    
    # save the unshifted spectrum to a fits file
    save_unshifted = True
    if save_unshifted:
        target = sm.target.s
        target_wl = sm.target.w   
        new_header = use_header
        new_header.set('NRDU', 'YES','Normalized, resampled, deblazed, unshifted')
        data_hdu = fits.PrimaryHDU(target, new_header) 
        wl_hdu = fits.ImageHDU(target_wl)
        hdu = fits.HDUList([data_hdu, wl_hdu])
        hdu.writeto(unshifted_out_path + '/' +  id_name + '_NDRU.fits')    


HD210610_spectra_2
HD210610_spectra_3
HD210610_spectra_4
HD210610_spectra_5
HIP5336_spectra_0
HIP5336_spectra_1
HIP5336_spectra_2
HIP5336_spectra_3
HIP5336_spectra_4
HIP5336_spectra_5
HIP5336_spectra_6
HIP6379_spectra_0
HIP6379_spectra_1
HIP6379_spectra_2
HIP6379_spectra_3
HIP6379_spectra_4
HIP6379_spectra_5
HIP6379_spectra_6
HIP7078_spectra_0
HIP7078_spectra_1
HIP7078_spectra_2
HIP7078_spectra_3
HIP7078_spectra_4
HIP7078_spectra_5
HIP7078_spectra_6
HIP7513_spectra_0
HIP7513_spectra_1
HIP7513_spectra_2
HIP7513_spectra_3
HIP7576_spectra_0
HIP7576_spectra_1
HIP7576_spectra_2
HIP7576_spectra_3
HIP7576_spectra_4
HIP7576_spectra_5
HIP7576_spectra_6
HIP7576_spectra_8
HIP7734_spectra_0
HIP7734_spectra_1
HIP7734_spectra_2
HIP7734_spectra_3
HIP7734_spectra_4
HIP7734_spectra_5
HIP7981_spectra_0
HIP7981_spectra_1
HIP7981_spectra_2
HIP7981_spectra_3
HIP8543_spectra_0
HIP8543_spectra_1
HIP8543_spectra_2
HIP8543_spectra_3
HIP8903_spectra_0
HIP8903_spectra_1
HIP8903_spectra_2
HIP8903_spectra_3
HIP890

bstar_deblaze.ipynb:269: RuntimeWarning: invalid value encountered in true_divide
  "    #scale_factors /= np.percentile(np.sort(y),99)\n",
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: Mean of empty slice


HIP9716_spectra_0
HIP9716_spectra_1
HIP9716_spectra_2
HIP9716_spectra_3
HIP9716_spectra_4
HIP9716_spectra_5
HIP9716_spectra_6
HIP9884_spectra_0
HIP9884_spectra_1
HIP9884_spectra_2
HIP9884_spectra_3
HIP10644_spectra_0
HIP10644_spectra_1
HIP10644_spectra_2
HIP10644_spectra_3
HIP11090_spectra_0
HIP11090_spectra_1
HIP11090_spectra_2
HIP11090_spectra_3
HIP11090_spectra_4
HIP11090_spectra_5
HIP11759_spectra_0
HIP11759_spectra_1
HIP11759_spectra_2
HIP11759_spectra_3
HIP12097_spectra_0
HIP12097_spectra_1
HIP12097_spectra_2
HIP12097_spectra_3
HIP12097_spectra_4
HIP12097_spectra_5
HIP12390_spectra_0
HIP12390_spectra_1
HIP12390_spectra_2
HIP12390_spectra_3
HIP12390_spectra_4
HIP12390_spectra_5
